##Load Data

In [ ]:
import pandas as pd
import numpy as np

from math import radians, sin, cos, sqrt, atan2

In [ ]:
usps_df = pd.read_excel('/content/zips ex.xlsx')  #Zipcode, Latitude, Longitude
members_df = pd.read_excel('/content/MSA zip codes only.xlsx')
partner_zips = ['60601', '30301', '10001', '94105']

In [ ]:
#Haversine formula to compute distance in miles
def haversine(lon1, lat1, lon2, lat2):
    R = 3958.8
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

##Check Zips

In [ ]:
# --- Check and Clean ZIP Codes ---
usps_zips_set = set(usps_df['Zipcode'].astype(str))

# Member ZIP checks
members_df['ZC.MOW SERVICE_ZIP'] = members_df['ZC.MOW SERVICE_ZIP'].astype(str)
valid_member_df = members_df[members_df['ZC.MOW SERVICE_ZIP'].isin(usps_zips_set)]
missing_member_zips = set(members_df['ZC.MOW SERVICE_ZIP']) - usps_zips_set

# Partner ZIP checks
partner_zips = [str(z) for z in partner_zips]
valid_partner_zips = [z for z in partner_zips if z in usps_zips_set]
missing_partner_zips = set(partner_zips) - usps_zips_set

# Log missing ZIPs
pd.DataFrame({'Missing Member Zips': list(missing_member_zips)}).to_csv('missing_member_zips.csv', index=False)
pd.DataFrame({'Missing Partner Zips': list(missing_partner_zips)}).to_csv('missing_partner_zips.csv', index=False)

print(f"Members Valid: {len(valid_member_df)} | Missing: {len(missing_member_zips)}")
print(f"Partners Valid: {len(valid_partner_zips)} | Missing: {len(missing_partner_zips)}")


Members Valid: 22746 | Missing: 6041
Partners Valid: 3 | Missing: 1


In [ ]:
valid_partner_zips

['60601', '10001', '94105']

In [ ]:
#Change all datatypes to string
usps_df['Zipcode'] = usps_df['Zipcode'].astype(str)
valid_member_df['ZC.MOW SERVICE_ZIP'] = valid_member_df['ZC.MOW SERVICE_ZIP'].astype(str)

<ipython-input-59-65619ed55e65>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_member_df['ZC.MOW SERVICE_ZIP'] = valid_member_df['ZC.MOW SERVICE_ZIP'].astype(str)


In [ ]:
member_coords = usps_df[usps_df['Zipcode'].isin(valid_member_df['ZC.MOW SERVICE_ZIP'])]
partner_coords = usps_df[usps_df['Zipcode'].isin(valid_partner_zips)]

In [ ]:
partner_coords.head()

,Zipcode,Latitude,Longitude
2583,10001,40.75064,-73.99728
21100,60601,41.88527,-87.62197
31599,94105,37.79239,-122.39707


#Calculate Distance

In [ ]:
#Create distance matrix using broadcasting
partner_lats = partner_coords['Latitude'].values
partner_lons = partner_coords['Longitude'].values
member_lats = member_coords['Latitude'].values
member_lons = member_coords['Longitude'].values

#Expand dimensions for broadcasting
partner_lats_2d = partner_lats[:, np.newaxis]
partner_lons_2d = partner_lons[:, np.newaxis]

In [ ]:
#Calculate distances (Partner ZIPs vs Member ZIPs)
distances = haversine(partner_lons_2d, partner_lats_2d, member_lons, member_lats)

In [ ]:
#Build DataFrame
distance_matrix = pd.DataFrame(distances, index=partner_coords['Zipcode'], columns=member_coords['Zipcode'])

##Output

In [ ]:
#Closest member to each partner
#closest_member_to_partner = distance_matrix.idxmin(axis=1)

#Closest partner to each member (transpose and find min)
#closest_partner_to_member = distance_matrix.T.idxmin(axis=1)

In [ ]:
#Closest member to each partner
closest_member_zips = distance_matrix.idxmin(axis=1)
zip_to_org = valid_member_df.set_index('ZC.MOW SERVICE_ZIP')['OR.MOW ORG_CST_KEY'].to_dict()
closest_member_org_keys = closest_member_zips.map(zip_to_org)

closest_member_detail = pd.DataFrame({
    'Partner ZIP': closest_member_zips.index,
    'Closest Member ZIP': closest_member_zips.values,
    'Closest Member ORG_CST_KEY': closest_member_org_keys.values
})

In [ ]:
#Closest partner to each member
closest_partner_zips = distance_matrix.T.idxmin(axis=1)

closest_partner_detail = pd.DataFrame({
    'Member ZIP': closest_partner_zips.index,
    'Closest Partner ZIP': closest_partner_zips.values
})

In [ ]:
# Save outputs
closest_member_detail.to_csv('closest_member_to_partner_with_org.csv', index=False)
closest_partner_detail.to_csv('closest_partner_to_member.csv', index=False)
distance_matrix.to_csv('distance_matrix.csv')